This notebook provides three different strategies how incremental drift can be handled by applying explicit drift detectors. Four different drift detectors are implemented: ADWIN, HDDDM, STEPD, MK. However, in this notebook only ADWIN & HDDDM are used.
The strategies comprise: 
- 1) incremental training/updating of a model after a drift was detected 
- 2) training of a new model and discard old model
- 3) a combination of incremental training and new training after drift was detected (referred to as "Switching")

All strategies are applied with a custom feedforward MLP model which was trained to predict taxi demand in different areas in New York City at the same time. 

## Import Packages

In [22]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import datetime
from dateutil.relativedelta import relativedelta


import itertools

import os


#from tqdm import tqdm

#import keras specific functions for storing and loading models
from keras.models import Model

from keras.models import model_from_json



#load custom deep Models (MLP)
from complex_mlp import ComplexMLP

#import custom functions to store all kinds of results on disk:
import save_files_collection as sv_files

#import all functions to perform drift detection & retraining
import drift_retraining_collection as dft


## Load & prepare Data

In [4]:
#load data
TRAIN_PATH = '/media/...'
Store_PATH = '/media/...'
file_final = 'preprocessed_data.csv'

df_m = pd.read_csv(TRAIN_PATH + file_final, header=0)

#convert to datetime format:
df_m['date'] = pd.to_datetime(df_m['date'], utc=True)
df_m['date'] = df_m['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_m['date'] = pd.to_datetime(df_m['date'])
#df_m = df_m.set_index("date") -> set index later, since we need "date" column to find highest demand columns..
print(df_m.head())

                 date  1  2  3    4  5  6   7  8  9  ...  254  255  256  257  \
0 2009-01-01 05:00:00  0  0  0   91  0  0  30  0  0  ...    0   50   39    3   
1 2009-01-01 06:00:00  1  0  0  105  0  0  62  0  0  ...    0   77   67    5   
2 2009-01-01 07:00:00  0  0  0   96  0  0  79  0  0  ...    0   90   83    4   
3 2009-01-01 08:00:00  0  0  0   91  0  0  84  0  0  ...    0   54   77    3   
4 2009-01-01 09:00:00  2  0  0   82  0  0  85  0  1  ...    0   66   54    4   

   258  259  260  261  262  263  
0    1    0    3   52  127  326  
1    0    0   15   65  166  476  
2    0    0   19   39  125  460  
3    1    0   19   54   79  313  
4    0    0   13   24   47  224  

[5 rows x 264 columns]


In [5]:
'''filter areas with highest demand '''
#get time series with highest "demand patterns":

#function filters nlargest areas:
def get_nlargest_areas(nlargest, org_dataset = df_m):
    
    #get time series with highest "demand patterns":
    df_sum = org_dataset.copy(deep=True).drop(columns=["date"],axis=1)
    df_sum = df_sum.sum(axis=0,numeric_only=True)

    #store nlargest values:
    df_sum = df_sum.nlargest(nlargest) 
    idx_filter = list(df_sum.index.values)
    #append "date" column
    idx_filter.append("date")

    del df_sum
    
    #filter columns with largest values:
    ts_largest = org_dataset[idx_filter].copy(deep=True)
    ts_largest = ts_largest.set_index("date")

    #shift datetimeindex to use local NYC time not UTC:
    ts_largest.index = ts_largest.index.shift(-5,freq='H')

    return ts_largest


In [6]:
ts_20largest = get_nlargest_areas(20)
ts_10largest = get_nlargest_areas(10)

print('20 largest areas: ', ts_20largest.shape)
print('10 largest areas: ', ts_10largest.shape)


20 largest areas:  (83231, 20)
10 largest areas:  (83231, 10)


## Load Models from Disk (trained on 2009-2010)

### function to load models

In [5]:
def load_pretrained_model_from_disk(model_type):
    
    '''
    Returns pre-trained model from disk
    '''


    model_PATH = '/media/...'    
    #complex MLP:
    model_architecture_complex_MLP_PATH = '/media/...'   
    complex_MLP_model_file = 'complex_MLP_early_stopping_W168_20areas__y2012.json'
    complex_MLP_weights = 'complex_MLP_early_stopping_W168_20areas__y2012_weights.h5'
    
    
    #multivar LSTM without additional features:
    model_architecture_multivar_20 = '/media/...'   
    multivar_LSTM_file = 'Model_Architectures_multivar20/multivariate_stacked_lstm_non_st_model_2H_256_32_batch512_drop03_clip_norm_shuffle_scaling_tanh_W168_20largest_areas__y2012.json'
    multivar_LSTM_weights = 'multivariate_stacked_lstm_non_st_model_2H_256_32_batch512_drop03_clip_norm_shuffle_scaling_tanh_W168_20largest_areas__y2012_weights.h5'

    
    
    #multivar LSTM with additional features:
    #prepare files for models:
    model_architecture_multivar_20 = '/media/...'   
    multivar_LSTM_feat_file = 'Model_Architectures_multivar20/multivariate_stacked_lstm_non_st_model_2H_256_128_batch512_drop03_clip_norm_shuffle_scaling_tanh_encoded_lag_feats_added_W168_20largest_areas__y2012.json'
    multivar_LSTM_feat_weights = 'multivariate_stacked_lstm_non_st_model_2H_256_128_batch512_drop03_clip_norm_shuffle_scaling_tanh_encoded_lag_feats_added_W168_20largest_areas__y2012_weights.h5'

    
    
    #XGBoost 2year n_estimators=2000:
    multivar_XGBoost_2000_model_file = 'multivar_XGBoost_est2000_maxdep7_s1_2009_e12_2010_2year_train'
    model_architecture_multivar_XGBoost_path = '/media/...'   
    
    model_architecture_multivar_XGBoost_path_new = '/media/...'   
    
    #XGBoost 2year n_estimators=1000:
    multivar_XGBoost_1000_model_file = 'multivar_XGBoost_est1000_maxdep7_20largest_areas_s1_2009_e12_2010_2year_train.pickle.dat'

    
    instances_dict = {'SingleMLP': (),
                  'SingleLSTM': (),
                  'ComplexMLP': (complex_MLP_model_file, model_architecture_complex_MLP_PATH, complex_MLP_weights),
                  'MultivarLSTM': (multivar_LSTM_file, model_architecture_multivar_20, multivar_LSTM_weights),
                  'MultivarLSTM_lag_feat': (multivar_LSTM_feat_file, model_architecture_multivar_20, multivar_LSTM_feat_weights),
                  'MultivarXGBoost_2year_est2000': (multivar_XGBoost_2000_model_file, model_architecture_multivar_XGBoost_path),
                  'MultivarXGBoost_2year_est1000': (multivar_XGBoost_1000_model_file, model_architecture_multivar_XGBoost_path_new)

                 }
    
       
    if 'XGBoost' not in model_type:

        #load complexMLP model 20largest areas:
        json_file = open(model_PATH + instances_dict[model_type][0], 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        prediction_model = model_from_json(loaded_model_json)

        #load weights of best model:
        prediction_model.load_weights(instances_dict[model_type][1] + instances_dict[model_type][2])

    
    else:
        model_PATH = instances_dict[model_type][1]

        model_name = instances_dict[model_type][0]

        #load model:
        file_to_load = model_PATH + model_name

        #load model into dict:
        prediction_model = pickle.load(open(file_to_load, "rb"))
        
     
    
    return prediction_model



### helper function to call multiple instances of a class

In [12]:
#function needed to create new instances, otherwise same instances is used!!
def create_model_instance(model_type, n_epochs_init = 150):
    
    instances_dict = {'ComplexMLP': ComplexMLP(n_epochs = n_epochs_init),
                      
                     }
    
    return instances_dict[model_type]

    
    

## ADWIN

### Strategy: training of a new model if drift is detected & discarding of old model
- Params for test purpose only: number of epochs = 20 instead of 150, end of dataset: 2013

In [17]:
#set model_name based on used params:
model_name = 'complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_adwin'


#create instance of class:
cplxMLP_model_new = create_model_instance('ComplexMLP')

#update instance with model from disk:
prediction_model = load_pretrained_model_from_disk('ComplexMLP')
cplxMLP_model_new.load_model(prediction_model)

#set dataset for slicing:
ts_series_input = ts_20largest.copy()



#call function for drift detection & retraining:
retraining_results_tuple = dft.drift_detection_retraining(model_instance = cplxMLP_model_new, org_ts_series=ts_series_input, 
                                                    model_name = model_name, detector_type = 'ADWIN', 
                                                    update_retrain_switch=False, first_forecast_range_days=168,
                                                    n_epochs_retrain = 20, overwrite_params = True,
                                                    end_of_dataset_date = '2013-12-31 23:00:00',
                                                    verbosity = 2)
                               

#assign results:
all_MODELS_dict_MLP_adwin = retraining_results_tuple[0]
all_model_RESULTS_dict_MLP_adwin = retraining_results_tuple[1] 
all_detected_DATES_dict_MLP_adwin = retraining_results_tuple[2]
avg_rmse_resulst_all_adwin = retraining_results_tuple[3]
all_retraining_dates_adwin = retraining_results_tuple[4]
all_weight_update_dates_adwin = retraining_results_tuple[5]





# Very first predictions are made for next 168 days..
## Assigned Dates are double checked..
 ++ Number of days contained in train_set used for scaling:  729
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', '2011-01-01 00:00:00', None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_adwin ####
80660/80660 [==============================] - 2s 21us/step
Shape of org. dataset after shift:  (4033, 20)
20/20 [==============================] - 0s 42us/step
Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[63.25613309572718]
## area  237
Share of wrongly classified observations:  0.16265807091495166
## area  237
Share of wrongly classified observations:  0.16265807091495166
## area  161
Share of wrongly classified observations:  0.2392759732209273
## area  161
Share of wrongly classified observations:  0.23927597322092736
## area  230
Sh

## Change detected in area binary170, index: 1631
date: 2011-03-09 23:00:00
Drift detected at:  2011-03-09 23:00:00
>> Current Time:  23/01/2020 11:32:48
## ++ previous detected dates:  [Timestamp('2011-01-01 00:00:00'), Timestamp('2011-03-09 23:00:00')]
## ++ last training dates:  [Timestamp('2009-01-01 00:00:00')]
 ++ Number of days contained in train_set used for scaling/retraining:  730
#### Current dates: 
#### training_start_date:  2009-03-09 23:00:00
#### start_valid_set:  None
#### start_test_set:  None
### ### New Model is trained
selected years for training:  [Timestamp('2009-03-09 23:00:00'), Timestamp('2011-03-09 23:00:00')]
year_list given:  [Timestamp('2009-03-09 23:00:00'), Timestamp('2011-03-09 23:00:00'), None, None]
#### Train model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_adwin_count1__trainsize730_s3_2009_e3_2011__stepsize1__p12_2013 ####
>> Dates are assigned...
>date_valid:  None
>date_test:  N

Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[61.998517665137136]
## area  237
Share of wrongly classified observations:  0.1678729037952339
## area  237
Share of wrongly classified observations:  0.1678729037952339
## area  161
Share of wrongly classified observations:  0.2510150044130627
## area  161
Share of wrongly classified observations:  0.2510150044130627
## area  230
Share of wrongly classified observations:  0.24942630185348635
## area  230
Share of wrongly classified observations:  0.24942630185348633
## area  79
Share of wrongly classified observations:  0.24430714916151808
## area  79
Share of wrongly classified observations:  0.2443071491615181
## area  236
Share of wrongly classified observations:  0.15728155339805827
## area  236
Share of wrongly classified observations:  0.15728155339805824
## area  162
Share of wrongly classified observations:  0.22453662842012356
## area  162
Share of wrongly classified observations:  0.22453662842

# adjusted dates..
start_train_year:  2009-04-16 07:00:00
last_train_set_year:  2011-04-16 07:00:00
start_validation_set_year:  2011-04-16 07:00:00
start_test_set_year:  2011-04-16 07:00:00
end_validation_set_year:  2011-04-16 07:00:00
end_test_set_year:  2011-04-16 07:00:00
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data

## area  142
Share of wrongly classified observations:  0.21109586953208354
## area  107
Share of wrongly classified observations:  0.1800030483158055
## area  107
Share of wrongly classified observations:  0.18000304831580552
## area  163
Share of wrongly classified observations:  0.19387288523090995
## area  163
Share of wrongly classified observations:  0.19387288523090992
## area  68
Share of wrongly classified observations:  0.1970736168267032
## area  68
Share of wrongly classified observations:  0.19707361682670324
## area  239
Share of wrongly classified observations:  0.2014936747447036
## area  239
Share of wrongly classified observations:  0.20149367474470356
## area  164
Share of wrongly classified observations:  0.18533760097546104
## area  164
Share of wrongly classified observations:  0.18533760097546106
## area  141
Share of wrongly classified observations:  0.15622618503276942
## area  141
Share of wrongly classified observations:  0.1562261850327694
## area  249
Share

...

## area  163
Share of wrongly classified observations:  0.19865813560828227
## area  163
Share of wrongly classified observations:  0.19865813560828224
## area  68
Share of wrongly classified observations:  0.20167955211943478
## area  68
Share of wrongly classified observations:  0.2016795521194348
## area  239
Share of wrongly classified observations:  0.19412601084155334
## area  239
Share of wrongly classified observations:  0.19412601084155337
## area  164
Share of wrongly classified observations:  0.18737225628721232
## area  164
Share of wrongly classified observations:  0.1873722562872123
## area  141
Share of wrongly classified observations:  0.1543144050475429
## area  141
Share of wrongly classified observations:  0.15431440504754287
## area  249
Share of wrongly classified observations:  0.17595307917888559
## area  249
Share of wrongly classified observations:  0.17595307917888564
## area  138
Share of wrongly classified observations:  0.3845196836399183
## area  138
Share

Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
X_train shape of area237 before concat with other areas:  (16872, 203)
X_valid shape of area237 before concat with other areas:  (1, 203)
X_test shape of area237 before concat with other areas:  (1, 203)
y_train shape of area237 before concat with other areas:  (16872,)
y_valid shape of area237 before concat with other

## area  163
Share of wrongly classified observations:  0.20217457420924578
## area  163
Share of wrongly classified observations:  0.20217457420924576
## area  68
Share of wrongly classified observations:  0.2019084549878345
## area  68
Share of wrongly classified observations:  0.20190845498783455
## area  239
Share of wrongly classified observations:  0.19286040145985406
## area  239
Share of wrongly classified observations:  0.192860401459854
## area  164
Share of wrongly classified observations:  0.18727189781021902
## area  164
Share of wrongly classified observations:  0.18727189781021897
## area  141
Share of wrongly classified observations:  0.15309458637469586
## area  141
Share of wrongly classified observations:  0.15309458637469586
## area  249
Share of wrongly classified observations:  0.1754105839416058
## area  249
Share of wrongly classified observations:  0.17541058394160583
## area  138
Share of wrongly classified observations:  0.3810827250608273
## area  138
Share 

Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
Data is scaled...
Standardizing used...
X_train shape of area237 before concat with other areas:  (16872, 203)
X_valid shape of area237 before concat with other areas:  (1, 203)
X_test shape of area237 before concat with other areas:  (1, 203)
y_train shape of area237 before concat with other areas:  (16872,)
y_valid shape of area237 before concat with other

## area  163
Share of wrongly classified observations:  0.20209854014598538
## area  163
Share of wrongly classified observations:  0.2020985401459854
## area  68
Share of wrongly classified observations:  0.2018324209245742
## area  68
Share of wrongly classified observations:  0.2018324209245742
## area  239
Share of wrongly classified observations:  0.1933546228710462
## area  239
Share of wrongly classified observations:  0.19335462287104624
## area  164
Share of wrongly classified observations:  0.1876140510948905
## area  164
Share of wrongly classified observations:  0.18761405109489052
## area  141
Share of wrongly classified observations:  0.15320863746958635
## area  141
Share of wrongly classified observations:  0.15320863746958638
## area  249
Share of wrongly classified observations:  0.17598083941605835
## area  249
Share of wrongly classified observations:  0.17598083941605838
## area  138
Share of wrongly classified observations:  0.3818810827250608
## area  138
Share o

### Strategy: incremental training/updating of model if drift is detected
- Params for test purpose only: number of epochs = 10 instead of 150, end of dataset: 2011, verbosity = 0

In [19]:
#set model_name based on used params:
model_name = 'complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_w_update_adwin'


#create instance of class:
cplxMLP_model_new = create_model_instance('ComplexMLP')

#update instance with model from disk:
prediction_model = load_pretrained_model_from_disk('ComplexMLP')
cplxMLP_model_new.load_model(prediction_model)

#set dataset for slicing:
ts_series_input = ts_20largest.copy()



#call function for drift detection & retraining:
retraining_results_tuple_w_update_adwin = dft.drift_detection_retraining(model_instance = cplxMLP_model_new, org_ts_series=ts_series_input, 
                                                    model_name = model_name, detector_type = 'ADWIN', 
                                                    update_retrain_switch=False, first_forecast_range_days=168,
                                                    n_epochs_weight = 10, overwrite_params = True,
                                                    end_of_dataset_date = '2011-12-31 23:00:00',
                                                    update_weights_flag = True,
                                                    verbosity = 0)
                               



# Very first predictions are made for next 168 days..
## Assigned Dates are double checked..
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', '2011-01-01 00:00:00', None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_w_update_adwin ####
80660/80660 [==============================] - 3s 32us/step
Shape of org. dataset after shift:  (4033, 20)
20/20 [==============================] - 0s 54us/step
Shape of org. dataset after shift:  (1, 20)
## Start Drift Detection
new detectors are created for each area...
## Change detected in area binary170, index: 1631
date: 2011-03-09 23:00:00
Drift detected at:  2011-03-09 23:00:00
>> Current Time:  23/01/2020 12:23:40
### ### New Model is trained
selected years for training:  [Timestamp('2009-03-09 23:00:00'), Timestamp('2011-03-09 23:00:00')]
year_list given:  [Timestamp('2009-03-09 23:00:00'), Timestamp('2011-03-09 23:00:00'), None,

3380/3380 [==============================] - 0s 25us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 49us/step
Shape of org. dataset after shift:  (1, 20)
## Start Drift Detection
pre-defined detectors are used...
> No drifts detected!
Make preds for next 7 days...
## Assigned Dates are double checked..
selected years for training:  [Timestamp('2009-04-21 15:00:00'), Timestamp('2011-04-21 15:00:00')]
year_list given:  [Timestamp('2009-04-21 15:00:00'), Timestamp('2011-04-21 15:00:00'), Timestamp('2011-10-27 20:00:00'), None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_w_update_adwin ####
3380/3380 [==============================] - 0s 24us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 49us/step
Shape of org. dataset after shift:  (1, 20)
## Start Drift Detection
pre-defined detectors are used...
> No drifts 

### Strategy: Switching Scheme 
- Params: backshift = 2, weight_range == True
- Params for test purpose only: number of epochs = 10 instead of 150, end of dataset: 2011

In [20]:
#Apply switching of retraining scheme: update weights & retrain model

#set model_name based on used params:
model_name = 'complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_switch_backshift2_weight_range_adwin'


#create instance of class:
cplxMLP_model_new = create_model_instance('ComplexMLP')

#update instance with model from disk:
prediction_model = load_pretrained_model_from_disk('ComplexMLP')
cplxMLP_model_new.load_model(prediction_model)

#set dataset for slicing:
ts_series_input = ts_20largest.copy()



#call function for drift detection & retraining:
retraining_results_tuple_adwin_switch_backshfit2_weight_range = dft.drift_detection_retraining(model_instance = cplxMLP_model_new, org_ts_series=ts_series_input, 
                                                    model_name = model_name, detector_type = 'ADWIN', 
                                                    update_retrain_switch=True, first_forecast_range_days=168,
                                                    n_epochs_retrain = 10, n_epochs_weight = 10, 
                                                    overwrite_params = True,
                                                    weight_update_backshift=2,
                                                    end_of_dataset_date = '2011-12-31 23:00:00',                
                                                    make_preds_with_weight_range = True,
                                                    verbosity = 2)
                               

                                                                                         


# Very first predictions are made for next 168 days..
## Assigned Dates are double checked..
 ++ Number of days contained in train_set used for scaling:  729
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', '2011-01-01 00:00:00', None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_switch_backshift2_weight_range_adwin ####
80660/80660 [==============================] - 3s 32us/step
Shape of org. dataset after shift:  (4033, 20)
20/20 [==============================] - 0s 52us/step
Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[63.25613309572718]
## area  237
Share of wrongly classified observations:  0.16265807091495166
## area  237
Share of wrongly classified observations:  0.16265807091495166
## area  161
Share of wrongly classified observations:  0.2392759732209273
## area  161
Share of wrongly classified observations:  0.23

## Change detected in area binary170, index: 1631
date: 2011-03-09 23:00:00
Drift detected at:  2011-03-09 23:00:00
>> Current Time:  23/01/2020 12:31:04
 ->> update_weights_flag set to "True" , delta of drift dates: -68
 >> delta of last start trainset & current drift:  -798
## ++ previous detected dates:  [Timestamp('2011-01-01 00:00:00'), Timestamp('2011-03-09 23:00:00')]
## ++ last training dates:  [Timestamp('2009-01-01 00:00:00')]
 ++ Number of days contained in train_set used for scaling/retraining:  759
#### Current dates: 
#### training_start_date:  2009-02-09 22:00:00
#### start_valid_set:  None
#### start_test_set:  None
### ### Model weights are updated based on Switching Scheme
selected years for training:  [Timestamp('2009-02-09 22:00:00'), Timestamp('2011-03-09 23:00:00')]
year_list given:  [Timestamp('2009-02-09 22:00:00'), Timestamp('2011-03-09 23:00:00'), None, None]
#### Train model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_a

## area  141
Share of wrongly classified observations:  0.14510150044130626
## area  249
Share of wrongly classified observations:  0.16716681376875553
## area  249
Share of wrongly classified observations:  0.16716681376875553
## area  138
Share of wrongly classified observations:  0.3754633715798764
## area  138
Share of wrongly classified observations:  0.3754633715798764
## area  90
Share of wrongly classified observations:  0.15922330097087378
## area  90
Share of wrongly classified observations:  0.15922330097087378
## converted_stream_flag used:  True
## Shape of streaming_df:  (4033, 20)
## Head of streaming_df:                       binary237  binary161  binary230  binary79  binary236  \
date                                                                        
2011-03-10 00:00:00          1          0          0         1          1   
2011-03-10 01:00:00          1          0          0         1          1   
2011-03-10 02:00:00          1          1          1         0 

final concatenated shape of X_train :  (350420, 203)
#Clipping Norm applied
Epoch 1/10
350420/350420 [==============================] - 6s 17us/step - loss: 0.2059 - mean_absolute_error: 0.3258
Epoch 2/10
350420/350420 [==============================] - 5s 14us/step - loss: 0.2052 - mean_absolute_error: 0.3258
Epoch 3/10
350420/350420 [==============================] - 5s 14us/step - loss: 0.2050 - mean_absolute_error: 0.3256
Epoch 4/10
350420/350420 [==============================] - 5s 14us/step - loss: 0.2040 - mean_absolute_error: 0.3251
Epoch 5/10
350420/350420 [==============================] - 5s 14us/step - loss: 0.2048 - mean_absolute_error: 0.3259
Epoch 6/10
350420/350420 [==============================] - 5s 14us/step - loss: 0.2045 - mean_absolute_error: 0.3256
Epoch 7/10
350420/350420 [==============================] - 5s 14us/step - loss: 0.2047 - mean_absolute_error: 0.3257
Epoch 8/10
350420/350420 [==============================] - 5s 14us/step - loss: 0.2038 - mean_abs

## Change detected in area binary162, index: 3839
date: 2011-10-11 23:00:00
Drift detected at:  2011-10-11 23:00:00
>> Current Time:  23/01/2020 12:34:05
 ->> update_weights_flag set to "True" , delta of drift dates: -160
 >> delta of last start trainset & current drift:  -1014
## ++ previous detected dates:  [Timestamp('2011-05-04 23:00:00'), Timestamp('2011-10-11 23:00:00')]
## ++ last training dates:  [Timestamp('2009-01-01 00:00:00')]
 ++ Number of days contained in train_set used for scaling/retraining:  759
#### Current dates: 
#### training_start_date:  2009-09-13 22:00:00
#### start_valid_set:  None
#### start_test_set:  None
### ### Model weights are updated based on Switching Scheme
selected years for training:  [Timestamp('2009-09-13 22:00:00'), Timestamp('2011-10-11 23:00:00')]
year_list given:  [Timestamp('2009-09-13 22:00:00'), Timestamp('2011-10-11 23:00:00'), None, None]
#### Train model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest

## area  164
Share of wrongly classified observations:  0.1882420091324201
## area  141
Share of wrongly classified observations:  0.14737442922374433
## area  141
Share of wrongly classified observations:  0.1473744292237443
## area  249
Share of wrongly classified observations:  0.17500000000000004
## area  249
Share of wrongly classified observations:  0.175
## area  138
Share of wrongly classified observations:  0.37385844748858443
## area  138
Share of wrongly classified observations:  0.3738584474885845
## area  90
Share of wrongly classified observations:  0.1670091324200913
## area  90
Share of wrongly classified observations:  0.16700913242009133
## converted_stream_flag used:  True
## Shape of streaming_df:  (1944, 20)
## Head of streaming_df:                       binary237  binary161  binary230  binary79  binary236  \
date                                                                        
2011-10-12 00:00:00          0          1          0         1          1   
2011

### Strategy: Switching Scheme 
- Params: backshift = 2, weight_range == True , weight_range_update = 4
- Params for test purpose only: number of epochs = 10 instead of 150, end of dataset: 2011

In [24]:
#Apply switching of retraining scheme: update weights & retrain model

#set model_name based on used params:
model_name = 'complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_switch_backshift2_weight_range_w_update_r4_adwin'


#create instance of class:
cplxMLP_model_new = create_model_instance('ComplexMLP')

#update instance with model from disk:
prediction_model = load_pretrained_model_from_disk('ComplexMLP')
cplxMLP_model_new.load_model(prediction_model)

#set dataset for slicing:
ts_series_input = ts_20largest.copy()



#call function for drift detection & retraining:
retraining_results_tuple_adwin_switch_backshfit2_weight_range_w_update_r4 = dft.drift_detection_retraining(model_instance = cplxMLP_model_new, org_ts_series=ts_series_input, 
                                                    model_name = model_name, detector_type = 'ADWIN', 
                                                    update_retrain_switch=True, first_forecast_range_days=168,
                                                    n_epochs_retrain = 10, n_epochs_weight = 10,
                                                    overwrite_params = True,
                                                    end_of_dataset_date = '2011-12-31 23:00:00',  
                                                    weight_update_backshift=2,
                                                    weight_update_range = [4],
                                                    adjust_lags_flag = True,
                                                    make_preds_with_weight_range = True,
                                                    verbosity = 0)
      

# Very first predictions are made for next 168 days..
## Assigned Dates are double checked..
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', '2011-01-01 00:00:00', None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_switch_backshift2_weight_range_w_update_r4_adwin ####
80660/80660 [==============================] - 3s 37us/step
Shape of org. dataset after shift:  (4033, 20)
20/20 [==============================] - 0s 65us/step
Shape of org. dataset after shift:  (1, 20)
## Start Drift Detection
new detectors are created for each area...
## Change detected in area binary170, index: 1631
date: 2011-03-09 23:00:00
Drift detected at:  2011-03-09 23:00:00
>> Current Time:  23/01/2020 12:46:51
### ### Model weights are updated based on Switching Scheme
selected years for training:  [Timestamp('2009-02-09 22:00:00'), Timestamp('2011-03-09 23:00:00')]
year_list given:

> No drifts detected!
Make preds for next 7 days...
## Assigned Dates are double checked..
selected years for training:  [Timestamp('2009-04-08 06:00:00'), Timestamp('2011-05-06 07:00:00')]
year_list given:  [Timestamp('2009-04-08 06:00:00'), Timestamp('2011-05-06 07:00:00'), Timestamp('2011-11-04 11:00:00'), None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_switch_backshift2_weight_range_w_update_r4_adwin ####
3380/3380 [==============================] - 0s 26us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 61us/step
Shape of org. dataset after shift:  (1, 20)
## Start Drift Detection
pre-defined detectors are used...
> No drifts detected!
Make preds for next 7 days...
## Assigned Dates are double checked..
selected years for training:  [Timestamp('2009-04-08 06:00:00'), Timestamp('2011-05-06 07:00:00')]
year_list given:  [Timestamp('2009-04-

#### store results

In [28]:
#call function to store results:
df_save_PATH = 'media/...'
model_save_PATH = 'media/...'

#call function to store results:
_ = sv_files.store_retrained_drift_detection_results(retraining_results_tuple_adwin_switch_backshfit2_weight_range_w_update_r4[1], 
                                            retraining_results_tuple_adwin_switch_backshfit2_weight_range_w_update_r4[0], 
                                            df_save_PATH)



#call function to store models & history:
#call function to store model:
_ = sv_files.store_model_and_history_on_disk(retraining_results_tuple_adwin_switch_backshfit2_weight_range_w_update_r4[0], 
                                model_save_PATH, df_save_PATH)



## store dates:
#call function to store dates at which change was detected:
dates_df = sv_files.store_detected_change_dates(retraining_results_tuple_adwin_switch_backshfit2_weight_range_w_update_r4[2],
                                           df_save_PATH)



## store dates with dates as index:
_ = sv_files.store_detected_change_dates_with_index(retraining_results_tuple_adwin_switch_backshfit2_weight_range_w_update_r4[2],
                                           retraining_results_tuple_adwin_switch_backshfit2_weight_range_w_update_r4[4],
                                           switch_updating_dates_list=retraining_results_tuple_adwin_switch_backshfit2_weight_range_w_update_r4[5],
                                           df_store_PATH=df_save_PATH)



predictions stored on disk!
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on di

## HDDDM Diff

### Strategy: training of new model if drift is detected
- Params for test purpose only: number of epochs = 10 instead of 150, end of dataset: 2011

In [25]:
#set model_name based on used params:
model_name = 'complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_hdddm_diff'



#create instance of class:
cplxMLP_model_new = create_model_instance('ComplexMLP')

#update instance with model from disk:
prediction_model = load_pretrained_model_from_disk('ComplexMLP')
cplxMLP_model_new.load_model(prediction_model)

#set dataset for slicing:
ts_series_input = ts_20largest.copy()



#call function for drift detection & retraining:
retraining_results_tuple_hdddm_diff = dft.drift_detection_retraining(model_instance = cplxMLP_model_new, org_ts_series=ts_series_input, 
                                                    model_name = model_name, detector_type = 'HDDDM_diff', 
                                                    use_differenced_ts=True,
                                                    update_retrain_switch=False, first_forecast_range_days=168,
                                                    n_epochs_retrain = 10, 
                                                    overwrite_params = True,
                                                    end_of_dataset_date = '2011-12-31 23:00:00', 
                                                    verbosity = 0)
                               

#assign results:
all_MODELS_dict_MLP_hdddm_diff = retraining_results_tuple_hdddm_diff[0]
all_model_RESULTS_dict_MLP_hdddm_diff = retraining_results_tuple_hdddm_diff[1] 
all_detected_DATES_dict_MLP_hdddm_diff = retraining_results_tuple_hdddm_diff[2]
avg_rmse_resulst_all_hdddm_diff = retraining_results_tuple_hdddm_diff[3]
all_retraining_dates_hdddm_diff = retraining_results_tuple_hdddm_diff[4]
all_weight_update_dates_hdddm_diff = retraining_results_tuple_hdddm_diff[5]





## >> Streaming data is differenced...
# Very first predictions are made for next 168 days..
## Assigned Dates are double checked..
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', '2011-01-01 00:00:00', None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_hdddm_diff ####
80660/80660 [==============================] - 3s 36us/step
Shape of org. dataset after shift:  (4033, 20)
20/20 [==============================] - 0s 91us/step
Shape of org. dataset after shift:  (1, 20)
## Start Drift Detection
new detectors are created for each area...


/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


> No drifts detected!
Make preds for next 7 days...
## Assigned Dates are double checked..
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', Timestamp('2011-06-18 01:00:00'), None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_hdddm_diff ####
3380/3380 [==============================] - 0s 23us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 66us/step
Shape of org. dataset after shift:  (1, 20)
## Start Drift Detection
pre-defined detectors are used...
> No drifts detected!
Make preds for next 7 days...
## Assigned Dates are double checked..
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', Timestamp('2011-06-25 02:00:00'), None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_retraining_hdddm_diff ####
3

create MLP Model:
#Dropout applied
#Clipping Norm applied
Train on 336960 samples, validate on 20 samples
Epoch 1/10
336960/336960 [==============================] - 6s 19us/step - loss: 0.3620 - mean_absolute_error: 0.4329 - val_loss: 0.6251 - val_mean_absolute_error: 0.6029
#Current LearningRate:  0.001
Epoch 2/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2830 - mean_absolute_error: 0.3826 - val_loss: 0.6043 - val_mean_absolute_error: 0.5744
#Current LearningRate:  0.001
Epoch 3/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2679 - mean_absolute_error: 0.3710 - val_loss: 0.5236 - val_mean_absolute_error: 0.5288
#Current LearningRate:  0.001
Epoch 4/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2586 - mean_absolute_error: 0.3641 - val_loss: 0.6315 - val_mean_absolute_error: 0.5662
#Current LearningRate:  0.001
Epoch 5/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2519 - 

/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Strategy: incremental training/updating of model if drift is detected
- Params for test purpose only: number of epochs = 10 instead of 150, end of dataset: 2011

In [26]:
#set model_name based on used params:
model_name = 'complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_weight_update_hdddm_diff'



#create instance of class:
cplxMLP_model_new = create_model_instance('ComplexMLP')

#update instance with model from disk:
prediction_model = load_pretrained_model_from_disk('ComplexMLP')
cplxMLP_model_new.load_model(prediction_model)

#set dataset for slicing:
ts_series_input = ts_20largest.copy()



#call function for drift detection & retraining:
weight_update_results_tuple_hdddm_diff = dft.drift_detection_retraining(model_instance = cplxMLP_model_new, org_ts_series=ts_series_input, 
                                                    model_name = model_name, detector_type = 'HDDDM_diff', 
                                                    use_differenced_ts=True,
                                                    update_weights_flag = True,
                                                    first_forecast_range_days=168,
                                                    n_epochs_weight = 10,
                                                    overwrite_params = True,
                                                    end_of_dataset_date = '2011-12-31 23:00:00', 
                                                    verbosity = 0)
                               




## >> Streaming data is differenced...
# Very first predictions are made for next 168 days..
## Assigned Dates are double checked..
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', '2011-01-01 00:00:00', None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_weight_update_hdddm_diff ####
80660/80660 [==============================] - 3s 37us/step
Shape of org. dataset after shift:  (4033, 20)
20/20 [==============================] - 0s 58us/step
Shape of org. dataset after shift:  (1, 20)
## Start Drift Detection
new detectors are created for each area...


/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


> No drifts detected!
Make preds for next 7 days...
## Assigned Dates are double checked..
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', Timestamp('2011-06-18 01:00:00'), None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_weight_update_hdddm_diff ####
3380/3380 [==============================] - 0s 24us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 58us/step
Shape of org. dataset after shift:  (1, 20)
## Start Drift Detection
pre-defined detectors are used...
> No drifts detected!
Make preds for next 7 days...
## Assigned Dates are double checked..
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', Timestamp('2011-06-25 02:00:00'), None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_weight_update_hdddm_diff 

#Clipping Norm applied
Epoch 1/10
336960/336960 [==============================] - 6s 18us/step - loss: 0.2113 - mean_absolute_error: 0.3299
Epoch 2/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2100 - mean_absolute_error: 0.3291
Epoch 3/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2087 - mean_absolute_error: 0.3284
Epoch 4/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2085 - mean_absolute_error: 0.3279
Epoch 5/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2076 - mean_absolute_error: 0.3276
Epoch 6/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2069 - mean_absolute_error: 0.3274
Epoch 7/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2065 - mean_absolute_error: 0.3268
Epoch 8/10
336960/336960 [==============================] - 5s 14us/step - loss: 0.2061 - mean_absolute_error: 0.3267
Epoch 9/10
336960/336960 [=======

/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Strategy: Switching Scheme 
- Params: backshift = 2, weight range == True, weight_update_range = [1,4.5]
- Params for test purpose only: number of epochs = 10 instead of 150, end of dataset: 2011

In [28]:
#set model_name based on used params:
model_name = 'complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_switch_backshift2_weight_range_w_update_r1__4_5_hdddm_diff'



#create instance of class:
cplxMLP_model_new = create_model_instance('ComplexMLP')

#update instance with model from disk:
prediction_model = load_pretrained_model_from_disk('ComplexMLP')
cplxMLP_model_new.load_model(prediction_model)

#set dataset for slicing:
ts_series_input = ts_20largest.copy()



#call function for drift detection & retraining:
results_tuple_switch_backshift2_weight_range_w_updater1__4_5_hdddm_diff = dft.drift_detection_retraining(model_instance = cplxMLP_model_new, org_ts_series=ts_series_input, 
                                                    model_name = model_name, detector_type = 'HDDDM_diff', 
                                                    use_differenced_ts=True,
                                                    update_retrain_switch=True, first_forecast_range_days=168,
                                                    make_preds_with_weight_range = True,
                                                    weight_update_backshift=2,
                                                    weight_update_range = [3,4.5],
                                                    n_epochs_retrain = 10, n_epochs_weight = 10,
                                                    overwrite_params = True,
                                                    end_of_dataset_date = '2012-12-31 23:00:00', 
                                                    verbosity = 2)
                               




## >> Streaming data is differenced...
# Very first predictions are made for next 168 days..
## Assigned Dates are double checked..
 ++ Number of days contained in train_set used for scaling:  729
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', '2011-01-01 00:00:00', None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_switch_backshift2_weight_range_w_update_r1__4_5_hdddm_diff ####
80660/80660 [==============================] - 3s 38us/step
Shape of org. dataset after shift:  (4033, 20)
20/20 [==============================] - 0s 71us/step
Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[63.25613309572718]
## converted_stream_flag used:  False
## Shape of streaming_df:  (4033, 20)
## Head of streaming_df:                         237    161    230     79    236    162    170   234  \
date                                                     

/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


> No drifts detected!
Make preds for next 7 days...
## Assigned Dates are double checked..
 ++ Number of days contained in train_set used for scaling:  729
selected years for training:  ['2009', '2010']
year_list given:  ['2009', '2010', Timestamp('2011-06-18 01:00:00'), None]
#### Make predictions model: complex_MLP_2H_128_32_batch512_drop03_clip_norm_shuffle_scaling_std_W168_20largest_areas_live_drift_switch_backshift2_weight_range_w_update_r1__4_5_hdddm_diff ####
3380/3380 [==============================] - 0s 25us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 66us/step
Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[55.150346178085684]
## converted_stream_flag used:  False
## Shape of streaming_df:  (169, 20)
## Head of streaming_df:                        237   161    230     79   236   162    170    234  \
date                                                                      
2011-06-18 01:00:0

3380/3380 [==============================] - 0s 25us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 62us/step
Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[61.881492602237145]
## converted_stream_flag used:  False
## Shape of streaming_df:  (169, 20)
## Head of streaming_df:                         237    161    230     79    236    162    170    234  \
date                                                                          
2011-07-02 03:00:00  -62.0  -48.0 -104.0 -473.0  -27.0  -44.0  -98.0  -17.0   
2011-07-02 04:00:00   -9.0    2.0  -23.0  -83.0   -7.0  -46.0  -20.0  -32.0   
2011-07-02 05:00:00    2.0    5.0   20.0  -27.0   22.0   12.0   82.0   15.0   
2011-07-02 06:00:00  105.0  200.0  164.0  -13.0  176.0  207.0  164.0  122.0   
2011-07-02 07:00:00  262.0  177.0   93.0  -88.0  259.0  121.0  224.0  142.0   

                        48    186    142    107   163     68    239    164  \
date   

3380/3380 [==============================] - 0s 23us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 80us/step
Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[57.588020147887164]
## converted_stream_flag used:  False
## Shape of streaming_df:  (169, 20)
## Head of streaming_df:                         237    161    230     79    236    162    170    234  \
date                                                                          
2011-07-23 06:00:00  -75.0 -144.0  -47.0  -49.0 -177.0  -67.0 -197.0  -33.0   
2011-07-23 07:00:00 -154.0 -144.0 -185.0  165.0    9.0  -80.0 -156.0 -140.0   
2011-07-23 08:00:00 -219.0 -211.0 -122.0  210.0   57.0 -277.0 -294.0 -185.0   
2011-07-23 09:00:00 -243.0 -109.0 -129.0  -42.0 -186.0 -272.0 -246.0 -168.0   
2011-07-23 10:00:00  -27.0 -130.0 -189.0  -62.0 -141.0  -34.0  -77.0 -137.0   

                        48    186   142    107   163    68    239    164  \
date     

...

3380/3380 [==============================] - 0s 26us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 68us/step
Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[71.3018936894146]
## converted_stream_flag used:  False
## Shape of streaming_df:  (169, 20)
## Head of streaming_df:                         237    161    230     79    236    162    170    234  \
date                                                                          
2012-06-30 19:00:00  -85.0    4.0  -50.0   -2.0 -145.0 -149.0  -83.0   15.0   
2012-06-30 20:00:00  -12.0 -124.0  102.0  367.0  -18.0   37.0 -154.0  102.0   
2012-06-30 21:00:00 -100.0    4.0  259.0   52.0 -107.0   -7.0 -197.0    0.0   
2012-06-30 22:00:00  -96.0 -105.0   17.0   27.0 -114.0 -118.0 -343.0    8.0   
2012-06-30 23:00:00  -98.0  -95.0   63.0   14.0  -35.0 -181.0 -362.0  -15.0   

                        48    186    142    107   163     68    239    164  \
date     

Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[61.85375616951668]
## converted_stream_flag used:  False
## Shape of streaming_df:  (169, 20)
## Head of streaming_df:                         237   161    230     79    236    162    170   234  \
date                                                                        
2012-07-14 21:00:00  -34.0 -27.0   -4.0  275.0 -288.0   37.0  -13.0  52.0   
2012-07-14 22:00:00 -118.0 -55.0  -89.0  232.0 -263.0 -112.0 -103.0 -89.0   
2012-07-14 23:00:00   -5.0 -39.0 -203.0  126.0  -82.0 -107.0  -63.0 -30.0   
2012-07-15 00:00:00   27.0 -54.0   58.0  362.0  -16.0 -120.0 -156.0  83.0   
2012-07-15 01:00:00   -1.0  13.0  168.0  318.0   -5.0   44.0   -9.0 -49.0   

                        48    186    142    107    163     68    239    164  \
date                                                                          
2012-07-14 21:00:00  -26.0  -12.0  253.0 -138.0   25.0  252.0 -214.0   45.0   
2012-07-14 22:00:00 -

3380/3380 [==============================] - 0s 25us/step
Shape of org. dataset after shift:  (169, 20)
20/20 [==============================] - 0s 66us/step
Shape of org. dataset after shift:  (1, 20)
## Avg. RMSE of recent predictions: 
[51.79386427960477]
## converted_stream_flag used:  False
## Shape of streaming_df:  (169, 20)
## Head of streaming_df:                        237   161   230     79   236   162    170   234     48  \
date                                                                           
2012-08-05 00:00:00  -4.0  55.0   0.0 -179.0  52.0  25.0  192.0  99.0  173.0   
2012-08-05 01:00:00  14.0 -85.0 -33.0 -248.0   2.0 -54.0   26.0  20.0  110.0   
2012-08-05 02:00:00  14.0  -1.0   4.0  -72.0  -7.0  18.0  -22.0  48.0   49.0   
2012-08-05 03:00:00 -23.0  -8.0  76.0   37.0  -8.0  27.0  -23.0   1.0   88.0   
2012-08-05 04:00:00  -7.0 -13.0 -23.0  -65.0  17.0  -7.0  -50.0 -25.0  -11.0   

                      186   142   107   163     68   239   164   141   249  \
d

#Current LearningRate:  0.001
Epoch 8/10
337440/337440 [==============================] - 5s 14us/step - loss: 0.2311 - mean_absolute_error: 0.3445 - val_loss: 0.1604 - val_mean_absolute_error: 0.3154
#Current LearningRate:  0.001
Epoch 9/10
337440/337440 [==============================] - 5s 14us/step - loss: 0.2282 - mean_absolute_error: 0.3425 - val_loss: 0.1667 - val_mean_absolute_error: 0.3191
#Current LearningRate:  0.001
Epoch 10/10
337440/337440 [==============================] - 5s 14us/step - loss: 0.2255 - mean_absolute_error: 0.3406 - val_loss: 0.1805 - val_mean_absolute_error: 0.3311
#Current LearningRate:  0.001
## Only training history & model are returned
## Predictions with retrained model are made..
>> Current Number of weight updates based on Switching Scheme:  1
>> Current Number of retrainings:  1
# Very first predictions are made for next 168 days..
## Assigned Dates are double checked..
# >> end of dataset is reached with preds of valid_set --> get last predictio

/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/vincent/anaconda3/envs/deeplearning_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#### store results

In [23]:
#call function to store results:
df_save_PATH = 'media/...'
model_save_PATH = 'media/...'

#call function to store results:
_ = sv_files.store_retrained_drift_detection_results(results_tuple_switch_backshift2_weight_range_w_updater5__4_5_hdddm_diff[1], 
                                            results_tuple_switch_backshift2_weight_range_w_updater5__4_5_hdddm_diff[0], 
                                            df_save_PATH)



#call function to store models & history:
#call function to store model:
_ = sv_files.store_model_and_history_on_disk(results_tuple_switch_backshift2_weight_range_w_updater5__4_5_hdddm_diff[0], 
                                model_save_PATH, df_save_PATH)



## store dates:
#call function to store dates at which change was detected:
dates_df = sv_files.store_detected_change_dates(results_tuple_switch_backshift2_weight_range_w_updater5__4_5_hdddm_diff[2],
                                           df_save_PATH)



## store dates with dates as index:
_ = sv_files.store_detected_change_dates_with_index(results_tuple_switch_backshift2_weight_range_w_updater5__4_5_hdddm_diff[2],
                                           results_tuple_switch_backshift2_weight_range_w_updater5__4_5_hdddm_diff[4],
                                           switch_updating_dates_list=results_tuple_switch_backshift2_weight_range_w_updater5__4_5_hdddm_diff[5],
                                           df_store_PATH=df_save_PATH)



predictions stored on disk!
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
Saved model to disk
Save history_df on disk done
